# Introduction
In this project, I am going to evaluate the current situation of the Madison metro transit ridership to see if it is reasonable to redesign the whole metro transit network. Also, going through the new proposed metro transit network including the new BRT system with the current bus stop ridership data to find out if the new BRT route would be a reasonable route for the community to have a more frequent and direct bus service.

Finally, there are lots of residents, students, and school faculties who are giving feedback to say that the new proposed routes will make them inconvenient to the bus stops. The issue includes problems with the parking area and the west service desert if they implement the new proposed routes. I would also review if these issues would be geographically true.

# Methods
### Visualization and Data Analysis 
#### (1) Folium (interactive map)

In this project, the data contains lots of information about the bus stops. Also, the extent of the metro transit covers the whole of Madison, which would be hard for the audience to see the detail. Therefore, I choose Folium to make an interactive map. Using an interactive map can allow everyone who is seeing this notebook to zoom in on where they are interested.

#### (2) pandas and GeoPandas

These two packages would not only allow me to do statistical analysis but use to convert data into geometry.

#### (3) OSMnx and NetworkX

For checking and comparing the BRT routes with the shortest path, these two packages would allow me to download the road network of Madison and combine the data layers I want to make a new map for my audience to easily recognize the routes in the real world.

#### (4) Matplotlib and Shapely

For some parts that don't exist in the data source or the part that I want to highlight, I can use Shapely to create Polygons, Points, and Polylines.

# Current Situation

## Current routes and stops

In [ ]:
# import modules
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import LineString, Point, Polygon
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from branca.element import Template, MacroElement
import math
import warnings

In [ ]:
# Mapbox api key
mpbox_key = "pk.eyJ1Ijoia2MtbWFwYm94IiwiYSI6ImNrcXJyMXVzNDBkaTYyb215dmZ0bTNsYXAifQ.Ii_D80VoG-PnuynT6P4SAA"

In [ ]:
# Open this cell to see the content(code) making current metro transit map
# Load Madison roads, and metro transit network
current_route_map = folium.Map(location=[43.080,-89.40], zoom_start=12,tiles='cartodbdark_matter', API_key=mpbox_key)
metro_route_gdf = gpd.read_file("Metro_Transit_Ridership_by_Route.shp")
for _, r in metro_route_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    folium.Popup(r['route_shor']).add_to(geo_j)
    geo_j.add_to(current_route_map)

# Load metro transit bus stop data
metro_stop_now = gpd.read_file('Metro_Transit_Ridership_by_Stop.shp')
stop_data_len = len(metro_stop_now)
for i in range(stop_data_len):
    # Add the message in the popup box, which shows up when the cursor moves onto a point.
    message = metro_stop_now['StopDescri'][i]+"(Click me for more information)"
    # Add ridership data into each point tooltip, which will be triggered when the user clicks on a point.
    ridership = "Ridership Data\nWeekday:%s\nSaturday:%s\nSunday:%s"%(metro_stop_now['Weekday'][i], metro_stop_now['Saturday'][i], metro_stop_now['Sunday'][i])
    folium.CircleMarker(
        location= [metro_stop_now['Lat'][i], metro_stop_now['Lon'][i]],          #location=[lat,lon]
        radius=1,
        color='tomato',
        fill=True,
        popup= ridership,
        tooltip=message
        ).add_to(current_route_map)
current_route_map

In [ ]:
# Checking Attributes
metro_stop_now.head()

### Ridership Analysis (by stops)
Calculate how many bus stops have their ridership above the average, and the percentage they occur in all stops.

In [ ]:
# Calculate mean value of ridership on weekdays, Saturday, and Sunday.
wkday_Ridership = []
sat_Ridership = []
sun_Ridership = []
for i in range(stop_data_len):
    wkday_rdsp = metro_stop_now['Weekday'][i]
    sat_rdsp = metro_stop_now['Saturday'][i]
    sun_rdsp = metro_stop_now['Sunday'][i]
    wkday_Ridership.append(wkday_rdsp)
    sat_Ridership.append(sat_rdsp)
    sun_Ridership.append(sun_rdsp)

mean_wkday = np.mean(wkday_Ridership)
mean_sat = np.mean(sat_Ridership)
mean_sun = np.mean(sun_Ridership)

# Count bus stops and calculate percentage.
# Collecting effective and ineffective bus stop index.
count = 0
wkday_count = 0
sat_count = 0
sun_count = 0
wkday_Gind = []
sat_Gind = []
sun_Gind = []
OneD_Gind = []
TwoD_Gind = []
ThreeD_Gind = []
BInd = []
for i in range(stop_data_len):
    if wkday_Ridership[i] >= mean_wkday:
        wkday_count += 1
        wkday_Gind.append(i)
    if sat_Ridership[i] >= mean_sat:
        sat_count += 1
        sat_Gind.append(i)
    if sun_Ridership[i] >= mean_sun:
        sun_count += 1
        sun_Gind.append(i)
    if wkday_Ridership[i] < mean_wkday:
        BInd.append(i)
    if sat_Ridership[i] < mean_sat:
        BInd.append(i)
    if sun_Ridership[i] < mean_sun:
        BInd.append(i)
    if wkday_Ridership[i] >= mean_wkday and sat_Ridership[i] < mean_sat and sun_Ridership[i] < mean_sun:
        OneD_Gind.append(i)
    elif wkday_Ridership[i] >= mean_wkday and sat_Ridership[i] >= mean_sat and sun_Ridership[i] >= mean_sun:
        ThreeD_Gind.append(i)
    elif wkday_Ridership[i] >= mean_wkday and sat_Ridership[i] >= mean_sat and sun_Ridership[i] < mean_sun:
        TwoD_Gind.append(i)
    elif wkday_Ridership[i] >= mean_wkday and sat_Ridership[i] < mean_sat and sun_Ridership[i] >= mean_sun:
        TwoD_Gind.append(i)
        
perc_1 = str(round(wkday_count/stop_data_len*100, 2))+"%"
perc_2 = str(round(sat_count/stop_data_len*100, 2))+"%"
perc_3 = str(round(sun_count/stop_data_len*100, 2))+"%"
print("There are %s bus stops are above average on weekdays."% (wkday_count)) 
print("There are %s bus stops are above average on Saturday."% (sat_count)) 
print("There are %s bus stops are above average on Sunday."% (sun_count)) 
print("The percentage of ridership above average on weekdays is "+perc_1)
print("The percentage of ridership above average on Saturday is "+perc_2)
print("The percentage of ridership above average on Sunday is "+perc_3)

##### Conclusion 1:

From this ridership data, we can learn that most of the bus stops are not used very effectively.

In [ ]:
# Open this cell to see the content(code) making the map about the bus stops that have ridership above average.
print("Current stops with ridership above average show on the following map.")
print("For bus stops, people are considered having different behavior on weekday, Saturday, and Sunday.")
print("If a bus stop is only effective on weekday, it is marked as WeekdayOnly.")
print("If a bus stop is effective on weekday and one of weekends, it is marked as Good.")
print("If a bus stop is effective on weekday and all of weekends, it is marked as Very Good.")
Above_avg_stop_map = folium.Map(location=[43.080,-89.40], zoom_start=12,tiles='cartodbdark_matter', API_key=mpbox_key)
for _, r in metro_route_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    style1 = {'fillColor': 'white', 'color': 'white', 'weight':1}
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x:style1)
    geo_j.add_to(Above_avg_stop_map)

# Add bus stops that are only above average on weekdays.
for i in OneD_Gind:
    message = metro_stop_now['StopDescri'][i]+"(WeekdayOnly)"
    folium.CircleMarker(
        location= [metro_stop_now['Lat'][i], metro_stop_now['Lon'][i]],          #location=[lat,lon]
        radius=1,
        color='tomato',
        fill=True,
        tooltip=message
        ).add_to(Above_avg_stop_map)

# Add bus stops that are above average on weekdays and one of weekends.
for i in TwoD_Gind:
    message = metro_stop_now['StopDescri'][i]+"(Good)"
    folium.CircleMarker(
        location= [metro_stop_now['Lat'][i], metro_stop_now['Lon'][i]],          #location=[lat,lon]
        radius=1,
        color='yellow',
        fill=True,
        tooltip=message
        ).add_to(Above_avg_stop_map)
        
# Add bus stops that are above average on weekdays and both weekends.
for i in ThreeD_Gind:
    message = metro_stop_now['StopDescri'][i]+"(Very Good)"
    folium.CircleMarker(
        location= [metro_stop_now['Lat'][i], metro_stop_now['Lon'][i]],          #location=[lat,lon]
        radius=1,
        color='lime',
        fill=True,
        tooltip=message
        ).add_to(Above_avg_stop_map)


In [ ]:
## Create Legned on Folium Map
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:tomato;opacity:0.7;'></span>WeekdayOnly</li>
    <li><span style='background:yellow;opacity:0.7;'></span>Good</li>
    <li><span style='background:lime;opacity:0.7;'></span>VeryGood</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

Above_avg_stop_map.get_root().add_child(macro)

##### Findings:

As we can see on the map, the bus stops on the far west side, the far southwest side, the far northwest side, and some around Capitol Square are only effective on weekdays. These bus stops are probably very important for people who use metro transit to work.

In [ ]:
# This cell can generate a list of bus stops that are above average.
print("If we want a list of bus stops that are above average for future research, uncomment the code in this cell.")
# stopID_lst = []
# stopName_lst = []
# lat_lst = []
# lon_lst = []
# note_lst = []
# for i in wkday_Gind:
#     stopID_lst.append(metro_stop_now['StopID'][i])
#     stopName_lst.append(metro_stop_now['StopDescri'][i])
#     lat_lst.append(metro_stop_now['Lat'][i])
#     lon_lst.append(metro_stop_now['Lon'][i])
#     note_lst.append("Weekday")
# for i in sat_Gind:
#     stopID_lst.append(metro_stop_now['StopID'][i])
#     stopName_lst.append(metro_stop_now['StopDescri'][i])
#     lat_lst.append(metro_stop_now['Lat'][i])
#     lon_lst.append(metro_stop_now['Lon'][i])
#     note_lst.append("Saturday")
# for i in sun_Gind:
#     stopID_lst.append(metro_stop_now['StopID'][i])
#     stopName_lst.append(metro_stop_now['StopDescri'][i])
#     lat_lst.append(metro_stop_now['Lat'][i])
#     lon_lst.append(metro_stop_now['Lon'][i])
#     note_lst.append("Sunday")


# dict = {"StopID": stopID_lst,"StopName": stopName_lst, "Latitude": lat_lst, "Longitude": lon_lst, "Note": note_lst}
# df = pd.DataFrame(data = dict)
# df.to_csv("BusStop_AboveAvg.csv", index=False)
# print("Bus stop List Generated")

# Proposed Metro Transit Route
In the meeting the metro transit representative says that the redesign project is going to mainly focus on increasing the ridership of buses, trying to reduce bus routes, and increasing the frequency of buses.


### The proposed metro transit network and BRT route.

In [ ]:
# Open this cell to see the content(code) making the proposed metro transit network map
print("The blue line is the proposed metro transit network, and the red line is the proposed BRT route.")
print("You can move your cursor on the map to see the label for the routes.")
draft_route_gdf = gpd.read_file("Draft_Network.shp")
draft_BRTroute_gdf = gpd.read_file("BRT_Draft_Route.shp")
draft_route_map = folium.Map(location=[43.080,-89.40], zoom_start=12,tiles='cartodbdark_matter', API_key=mpbox_key)

# Add draft metro transit network
for _, row in draft_route_gdf.iterrows():
    message="Blue: Metro Transit draft network"
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    style2 = {'fillColor': 'cornflowerblue', 'color': 'cornflowerblue', 'weight':1}
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x:style2,tooltip=message)
    geo_j.add_to(draft_route_map)

# Add bus stops that are only above average on weekdays.
for i in OneD_Gind:
    message = metro_stop_now['StopDescri'][i]+"(Weekday)"
    folium.CircleMarker(
        location= [metro_stop_now['Lat'][i], metro_stop_now['Lon'][i]],          #location=[lat,lon]
        radius=1,
        color='tomato',
        fill=True,
        tooltip=message
        ).add_to(draft_route_map)

# Add bus stops that are above average on weekdays and one of weekends.
for i in TwoD_Gind:
    message = metro_stop_now['StopDescri'][i]+"(Sat.)"
    folium.CircleMarker(
        location= [metro_stop_now['Lat'][i], metro_stop_now['Lon'][i]],          #location=[lat,lon]
        radius=1,
        color='yellow',
        fill=True,
        tooltip=message
        ).add_to(draft_route_map)

# Add bus stops that are above average on weekdays and both weekends.
for i in ThreeD_Gind:
    message = metro_stop_now['StopDescri'][i]+"(Sun.)"
    folium.CircleMarker(
        location= [metro_stop_now['Lat'][i], metro_stop_now['Lon'][i]],          #location=[lat,lon]
        radius=1,
        color='lime',
        fill=True,
        tooltip=message
        ).add_to(draft_route_map)

# Add proposed BRT route
for _, row in draft_BRTroute_gdf.iterrows():
    message="Red: BRT draft route."
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    style3 = {'fillColor': 'red', 'color': 'red', 'weight':3}
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x:style3,tooltip=message)
    geo_j.add_to(draft_route_map)

draft_route_map


### The shortest path

The proposed BRT route starts from S Junction Rd to East Springs Dr. The shortest path is shown below.

In [ ]:
# Open this cell to see the content(code) making the comparison map
print("This comparison map shows the proposed BRT route and the shortest path from the starting point to the endpoint of the proposed BRT route.")
print("The yellow line is the shortest path, and the red line is the proposed BRT route.")
warnings.simplefilter(action='ignore', category=FutureWarning)
G = ox.graph_from_place("Madison, Wisconsin, USA", network_type="drive")
# plot the street network with folium
osm_map = ox.plot_graph_folium(G, popup_attribute="name", weight=1, color="white",tiles='cartodbdark_matter', API_key=mpbox_key)

# find the nearest nodes of starting point and destination on OSMnx data
orig = ox.distance.nearest_nodes(G, -89.52761, 43.05748)
dest = ox.distance.nearest_nodes(G, -89.30306, 43.13140)
# use networkx to generate the shortest path from starting point to the endpoint.
route = nx.shortest_path(G, orig, dest)
# plot the route with folium on top of the previously created graph_map
comparison_map = ox.plot_route_folium(G, route, route_map=osm_map, fit_bounds=False, popup_attribute="length", weight=3, color="yellow", zoom=12, tiles='cartodbdark_matter', API_key=mpbox_key)

# Add proposed BRT route
for _, row in draft_BRTroute_gdf.iterrows():
    message="Red: BRT draft route."
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    style3 = {'fillColor': 'red', 'color': 'red', 'weight':3}
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x:style3,tooltip=message)
    geo_j.add_to(comparison_map)

comparison_map


##### Conclusion 2:

Through comparison, you can see the BRT route goes along most of the effective bus stops, which goes through Mineral Point Rd, Sheboygan Ave, University Ave, Campus Dr, Capitol Square to Washington Dr instead of just a very straightforward shortest path to go into downtown Madison.

# The main issue our community raised at the meeting.

In this section, we are going to review an issue that some residents, students and school faculties brought up in one of the online meeting of metro transit redesign.

### Park and transit in the park on the west side of Madison

The new proposed metro transit network is inconvenient for people to transfer from their car, and affect lots of the residents who have bought their house near bus routes.

Residents' concern: They bought their house with the consideration of the convenience of bus routes. The proposed route will have an impact on their daily life.

Students and school faculties' concern: The proposed bus routes detour, making them become inefficient for transfer.

In [ ]:
# Open this cell to see the content(code) making the map for the issue.
print("The red zone is the area that has issue.")
print("The yellow route did not exist in the meeting.")
print("The pinks are the platted lots, which are mostly free parking lots.")
draft_route_gdf = gpd.read_file("Draft_Network.shp")
park_gdf = gpd.read_file("Parks.shp")
draft_route_map_2 = folium.Map(location=[43.080,-89.40], zoom_start=12,tiles='cartodbdark_matter', API_key=mpbox_key)

# Add draft metro transit network
for _, row in draft_route_gdf.iterrows():
    message="Blue: Metro Transit draft network"
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    style2 = {'fillColor': 'cornflowerblue', 'color': 'cornflowerblue', 'weight':1}
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x:style2,tooltip=message)
    geo_j.add_to(draft_route_map_2)

# Add Parks 
for _, row in park_gdf.iterrows():
    message = "Park Area"
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    style4= {'fillColor': 'lime', 'color': 'grey', 'weight':1}
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x:style4,tooltip=message)
    geo_j.add_to(draft_route_map_2)

# Add west bus service desert polygon
lat_point_list = [43.10464, 43.10436, 43.10576, 43.10983, 43.11187, 43.09720, 43.09698, 43.07542, 43.07528, 43.06088, 43.06124, 43.06475, 43.06707, 43.07640, 43.09024]
lon_point_list = [-89.48702, -89.49285, -89.49742, -89.50283, -89.50950, -89.51147, -89.52355, -89.52502, -89.50739, -89.50767, -89.47249, -89.47115, -89.46835, -89.46757, -89.48590]
polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
style5 = {'fillColor': 'red', 'weight':1}
issueArea = folium.GeoJson(data=polygon_geom, style_function=lambda x:style5, tooltip="West service desert")
issueArea.add_to(draft_route_map_2)

# Add a height light for a bus route
coordinates = [
    [43.10337,-89.49236],
    [43.09881,-89.49398],
    [43.09705,-89.49552],
    [43.09720,-89.50170],
    [43.09179,-89.50191],
    [43.07641,-89.50255],
    [43.07423,-89.4675]]
added_Route=folium.PolyLine(locations=coordinates,weight=3, color='yellow', tooltip="This route is added according to the need of the public.")
added_Route.add_to(draft_route_map_2)

# Add platted lots in parks location
plat_lots_gdf = gpd.read_file("Platted_Lots_In_Parks_Lines.shp")
for _, row in plat_lots_gdf.iterrows():
    message="Platted lots in parks"
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    style6= {'fillColor': 'magenta', 'color': 'magenta', 'weight':1}
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x:style6,tooltip=message)
    geo_j.add_to(draft_route_map_2)


draft_route_map_2

##### Conclusion 3:

The meeting for UW students and employees was on April 14, and I got this draft network on April 18. We can see there are lots of parking lots in parks inside and near the red area, so we can know that the comment from attendees is reasonable. Also, the metro transit is likely going to add this new route into their new metro transit network in the future based on the feedback from the community.


# Conclusion and Discussion

In this project, we can learn that redesigning the metro transit system is reasonable because the percentage of effective bus stops is low. We can also learn that some of the bus stops may be important on the weekdays even though they are only effective on weekdays. Thus, some of those areas may need to be treated as different cases. For example, some of the bus routes may only be available on weekdays. 

For the new BRT design, we have confirmed that the BRT route is designed based on the ridership for serving as many people as possible instead of just a very direct route into downtown Madison. Thus, the proposed BRT route looks reasonable as well.

For the proposed network issue on the westside of Madison, near Middleton, we can see that a new route that did not show up in the meeting of April 14 has been added in this data that I got on April 18. This new route seems to effectively deal with the west service desert issue. This also means that the comments and feedback from the community are really being taken into consideration instead of being ignored. Therefore, I think the metro transit so far is doing a great job for this project.

### Discussion question:

This metro redesign project is mainly focusing on increasing Ridership, but will Coverage is more important than Ridership? Or, the metro transit system is only important on weekdays because people are mostly driving on weekends?

# Reference and Data Source

### Metro Transit Route - from City of Madison Open Data
    Metro_Transit_Ridership_by_Stop
    Metro_Transit_Ridership_by_Route
    Platted Lots In Parks Lines

### Metro Transit Draft Network Shapefiles - from Michael Smith, Map & Layout Designer, Metro Transit
    BRT_Stations
    Draft_Network
    

### CITY OF MADISON
##### Transit Network Redesign
    URL: https://www.cityofmadison.com/metro/routes-schedules/transit-network-redesign

##### Community Engagement (Previous Network Redesign Meetings)
    URL: https://www.cityofmadison.com/metro/routes-schedules/transit-network-redesign/community-engagement
